<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="##-Predicting-open-value-of-crude-oil-after-60-days-taking-past-data-into-account." data-toc-modified-id="#-Predicting-open-value-of-crude-oil-after-60-days-taking-past-data-into-account.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span># Predicting open value of crude oil after 60 days taking past data into account.</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import math
import sklearn
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 


# # Predicting open value of crude oil after 60 days taking past data into account.

In [10]:
df = pd.read_excel(r"D:/Research project/share data/08-08-2008 to 01-07-2016/Crude data.xlsx")

In [11]:
df

,Date,symbol,open,high,low,close,volume
0,2008-08-08,Crude Oil,119.900002,120.080002,114.620003,115.199997,297330
1,2008-08-10,Crude Oil,0.000000,0.000000,0.000000,0.000000,0
2,2008-08-11,Crude Oil,115.199997,116.900002,112.720001,114.449997,295334
3,2008-08-12,Crude Oil,114.529999,115.949997,112.309998,113.010002,303791
4,2008-08-13,Crude Oil,113.089996,117.459999,112.870003,116.000000,322967
...,...,...,...,...,...,...,...
2398,2016-06-26,Crude Oil,0.000000,0.000000,0.000000,0.000000,0
2399,2016-06-27,Crude Oil,47.810001,47.959999,45.830002,46.330002,504613
2400,2016-06-28,Crude Oil,46.590000,48.180000,46.540001,47.849998,498851
2401,2016-06-29,Crude Oil,48.060001,50.000000,47.980000,49.880001,511476


In [12]:
df.tail()

,Date,symbol,open,high,low,close,volume
2398,2016-06-26,Crude Oil,0.000000,0.000000,0.000000,0.000000,0
2399,2016-06-27,Crude Oil,47.810001,47.959999,45.830002,46.330002,504613
2400,2016-06-28,Crude Oil,46.590000,48.180000,46.540001,47.849998,498851
2401,2016-06-29,Crude Oil,48.060001,50.000000,47.980000,49.880001,511476
2402,2016-06-30,Crude Oil,49.549999,49.619999,48.169998,48.330002,508736


In [13]:
df.describe()


,open,high,low,close,volume
count,2403.000000,2403.000000,2403.000000,2403.000000,2403.000000
mean,64.610487,65.527095,63.577595,64.571573,254182.442780
std,36.343200,36.686851,35.935104,36.337536,161220.720416
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,43.510001,44.774999,42.055000,43.449999,175216.500000
50%,77.059998,78.040001,75.660004,77.000000,266350.000000
75%,94.715000,95.744999,93.625000,94.790001,340900.000000
max,121.550003,130.000000,115.639999,121.180000,869957.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2403 non-null   datetime64[ns]
 1   symbol  2403 non-null   object        
 2   open    2403 non-null   float64       
 3   high    2403 non-null   float64       
 4   low     2403 non-null   float64       
 5   close   2403 non-null   float64       
 6   volume  2403 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 131.5+ KB


In [28]:
data = df[df.symbol == 'Crude Oil']

In [29]:
data = data.drop(['symbol'], axis= 1)

In [30]:
data_training = data[data['Date']<'2015-01-01'].copy()
data_test = data[data['Date']>='2015-01-01'].copy()

In [32]:
data_training = data_training.drop(['Date'], axis = 1)


In [33]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)
data_training

array([[0.98642533, 0.92369232, 0.99117956, 0.9506519 , 0.44372181],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.94775808, 0.89923078, 0.97474924, 0.94446276, 0.44074307],
       ...,
       [0.45290002, 0.42876925, 0.45745419, 0.44239974, 0.36022158],
       [0.44195804, 0.41784615, 0.45572468, 0.44660834, 0.32616456],
       [0.44319208, 0.41553846, 0.4534763 , 0.43959399, 0.34848869]])

In [34]:
X_train = []
y_train = []

for i in range(60, data_training.shape[0]):
    X_train.append(data_training[i-60:i])
    y_train.append(data_training[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [35]:
X_train.shape

(1886, 60, 5)

In [36]:
X_train

array([[[0.98642533, 0.92369232, 0.99117956, 0.9506519 , 0.44372181],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.94775808, 0.89923078, 0.97474924, 0.94446276, 0.44074307],
        ...,
        [0.64788151, 0.60899998, 0.63602563, 0.61511801, 0.33001334],
        [0.6055121 , 0.57307692, 0.59296092, 0.57641523, 0.42247665],
        [0.59481696, 0.57153848, 0.60394324, 0.59291961, 0.21177557]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.94775808, 0.89923078, 0.97474924, 0.94446276, 0.44074307],
        [0.94224596, 0.89192305, 0.97120373, 0.93257965, 0.45336392],
        ...,
        [0.6055121 , 0.57307692, 0.59296092, 0.57641523, 0.42247665],
        [0.59481696, 0.57153848, 0.60394324, 0.59291961, 0.21177557],
        [0.        , 0.        , 0.        , 0.        , 0.        ]],

       [[0.94775808, 0.89923078, 0.97474924, 0.94446276, 0.44074307],
        [0.94224596, 0.89192305, 0.97120373, 0.93257965, 0.4

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [38]:
regressor = Sequential()

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = 'relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [39]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 60)            15840     
_________________________________________________________________
dropout (Dropout)            (None, 60, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 80)            45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 80)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               9

In [40]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs=50, batch_size=32)

Train on 1886 samples
Epoch 1/50
1886/1886 [==============================] - 11s 6ms/sample - loss: 0.1198
Epoch 2/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0824
Epoch 3/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0854
Epoch 4/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0845
Epoch 5/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0839
Epoch 6/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0829
Epoch 7/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0817
Epoch 8/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0814
Epoch 9/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0818
Epoch 10/50
1886/1886 [==============================] - 6s 3ms/sample - loss: 0.0819
Epoch 11/50
1886/1886 [==============================] - 7s 3ms/sample - loss: 0.0813
Epoch 12/50
1886/1886 [=================

In [41]:
data_test.head()

,Date,open,high,low,close,volume
1946,2015-01-02,53.759998,55.110001,52.029999,52.689999,268708
1947,2015-01-04,0.000000,0.000000,0.000000,0.000000,0
1948,2015-01-05,52.610001,52.730000,49.680000,50.040001,375782
1949,2015-01-06,50.000000,50.369999,47.549999,47.930000,451642
1950,2015-01-07,48.000000,49.310001,46.830002,48.650002,460083


In [42]:
data_training1 = data[data['Date']<'2015-01-01'].copy()


In [43]:
past_60_days = data_training1.tail(60)

In [44]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['Date'], axis = 1)
df.head()

,open,high,low,close,volume
0,82.760002,84.050003,82.290001,82.809998,297522
1,82.550003,83.150002,80.220001,80.519997,352217
2,80.419998,82.370003,80.050003,82.089996,354839
3,81.949997,81.949997,80.360001,81.010002,272509
4,0.000000,0.000000,0.000000,0.000000,0


In [45]:
inputs = scaler.transform(df)
inputs

array([[0.68087207, 0.64653848, 0.711605  , 0.68336357, 0.44400835],
       [0.67914439, 0.6396154 , 0.69370462, 0.66446606, 0.52563268],
       [0.6616207 , 0.63361541, 0.69223455, 0.67742198, 0.52954564],
       ...,
       [0.38329904, 0.37061538, 0.40245591, 0.39486712, 0.74446262],
       [0.39539284, 0.38461538, 0.41490834, 0.41161909, 0.7633036 ],
       [0.40765115, 0.3816923 , 0.41655135, 0.39882821, 0.75921454]])

In [46]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((457, 60, 5), (457,))

In [47]:
y_pred = regressor.predict(X_test)

In [48]:
scaler.scale_

array([8.22706685e-03, 7.69230769e-03, 8.64752688e-03, 8.25218683e-03,
       1.49235467e-06])

In [49]:
scale = 1/8.22706685e-03
scale

121.55000296369293

In [50]:
y_pred = y_pred*scale
y_test = y_test*scale

In [52]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score


In [53]:
k = mae(y_pred, y_test)
print('Training Mean Absolute Error', k )

Training Mean Absolute Error 7.675155558627326


In [54]:
sklearn.metrics.r2_score(y_test, y_pred)

0.7214529141989481

In [58]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test, y_pred)
mse

99.46944446541629

In [60]:
#RMSE
Rmse=np.sqrt(mse)
Rmse

9.97343694347221

In [61]:
# adjusted r2
var_test = y_test.var()
var_test
Adj_rsquare = 1-(mse/var_test)
Adj_rsquare

0.7214529141989481